<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/neural_networks/NSL/adversarial_regularization_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adversarial regularization for image classification

In [1]:
!pip install -q neural-structured-learning

     |████████████████████████████████| 112kB 18.3MB/s 


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import neural_structured_learning as nsl
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Hiperparametry

In [0]:
class HParams(object):
  def __init__(self):
    self.input_shape = [28, 28, 1]
    self.num_classes = 10
    self.conv_filters = [32, 64, 64]
    self.kernel_size = (3, 3)
    self.pool_size = (2, 2)
    self.num_fc_units = [64]
    self.batch_size = 32
    self.epochs = 5
    self.adv_multiplier = 0.2
    self.adv_step_size = 0.2
    self.adv_grad_norm = 'infinity'

HPARAMS = HParams()

### Zbiór danych MNIST

In [0]:
datasets = tfds.load('mnist')

train_dataset = datasets['train']
test_dataset = datasets['test']

IMAGE_INPUT_NAME = 'image'
LABEL_INPUT_NAME = 'label'

In [0]:
def normalize(features):
  features[IMAGE_INPUT_NAME] = tf.cast(
      features[IMAGE_INPUT_NAME], dtype=tf.float32) / 255.0
  return features

def convert_to_tuples(features):
  return features[IMAGE_INPUT_NAME], features[LABEL_INPUT_NAME]

def convert_to_dictionaries(image, label):
  return {IMAGE_INPUT_NAME: image, LABEL_INPUT_NAME: label}

train_dataset = train_dataset.map(normalize).shuffle(10000).batch(HPARAMS.batch_size).map(convert_to_tuples)
test_dataset = test_dataset.map(normalize).batch(HPARAMS.batch_size).map(convert_to_tuples)

### Model bazowy

In [0]:
def build_base_model(hparams):
  inputs = tf.keras.Input(
      shape=hparams.input_shape, dtype=tf.float32, name=IMAGE_INPUT_NAME)
  
  x = inputs
  for i, num_filters in enumerate(hparams.conv_filters):
    x = tf.keras.layers.Conv2D(
        num_filters, hparams.kernel_size, activation='relu')(x)
    if i < len(hparams.conv_filters) - 1:
      # max pooling między warstwami splotu
      x = tf.keras.layers.MaxPooling2D(hparams.pool_size)(x)
  x = tf.keras.layers.Flatten()(x)
  for num_units in hparams.num_fc_units:
    x = tf.keras.layers.Dense(num_units, activation='relu')(x)
  pred = tf.keras.layers.Dense(hparams.num_classes, activation='softmax')(x)
  model = tf.keras.Model(inputs=inputs, outputs=pred)
  return model

In [9]:
base_model = build_base_model(HPARAMS)
base_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0     

In [11]:
base_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                   metrics=['acc'])
base_model.fit(train_dataset, epochs=HPARAMS.epochs)

Train on None steps
Epoch 1/5
1875/1875 [==============================] - 25s 13ms/step - loss: 0.1426 - acc: 0.9550
Epoch 2/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0463 - acc: 0.9855
Epoch 3/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0334 - acc: 0.9896
Epoch 4/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0239 - acc: 0.9923
Epoch 5/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0197 - acc: 0.9940


In [12]:
results = base_model.evaluate(test_dataset)
named_results = dict(zip(base_model.metrics_names, results))
print('\naccuracy:', named_results['acc'])

    313/Unknown - 3s 10ms/step - loss: 0.0325 - acc: 0.9906
accuracy: 0.9906


### Adversarial-regularized model

In [0]:
adv_config = nsl.configs.make_adv_reg_config(
    multiplier = HPARAMS.adv_multiplier,
    adv_step_size = HPARAMS.adv_step_size,
    adv_grad_norm = HPARAMS.adv_grad_norm
)

In [0]:
base_adv_model = build_base_model(HPARAMS)
adv_model = nsl.keras.AdversarialRegularization(
    base_adv_model,
    label_keys = [LABEL_INPUT_NAME],
    adv_config = adv_config
)

train_set_for_adv_model = train_dataset.map(convert_to_dictionaries)
test_set_for_adv_model = test_dataset.map(convert_to_dictionaries)

In [18]:
adv_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                  metrics=['acc'])
adv_model.fit(train_set_for_adv_model, epochs=HPARAMS.epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on None steps
Epoch 1/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.2993 - sparse_categorical_crossentropy: 0.1377 - sparse_categorical_accuracy: 0.9599 - adversarial_loss: 0.8081
Epoch 2/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1227 - sparse_categorical_crossentropy: 0.0431 - sparse_categorical_accuracy: 0.9870 - adversarial_loss: 0.3981
Epoch 3/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0796 - sparse_categorical_crossentropy: 0.0325 - sparse_categorical_accuracy: 0.9897 - adversarial_loss: 0.2356
Epoch 4/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0547 - sparse_categorical_crossentropy: 0.0246 - sparse_categorical_accuracy: 0.9920 - adversarial_loss: 0.1504
Epoch 5/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0463 - sparse_categorical_crossentropy: 0.0184 - sparse_categorical_accuracy: 0.9942 - adversarial_loss: 0.1393


In [20]:
results = adv_model.evaluate(test_set_for_adv_model)
named_results = dict(zip(adv_model.metrics_names, results))
print('\naccuracy:', named_results['sparse_categorical_accuracy'])

    313/Unknown - 3s 11ms/step - loss: 0.0593 - sparse_categorical_crossentropy: 0.0343 - sparse_categorical_accuracy: 0.9895 - adversarial_loss: 0.1249
accuracy: 0.9895
